In [1]:
#UI
import tkinter as tk
from tkinter import ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

#SQL
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Text, String, DateTime

#Graph
import matplotlib.pyplot as plt
import pandas as pd

#Data Manipulation
from datetime import date

#Linear Regression
import seaborn as sns

#Clustering
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

#Classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay

In [2]:
def LoadCsv():
    df = pd.read_csv("Marketing_Campaign_Data.csv")
    DataCleaning(df)
    return df.dropna()

In [3]:
def DataCleaning(df):
    for i in df.index:
        if df.loc[i, "Education"] == "Graduation":
            df.loc[i, "Education"] = "Bachelor"
            
        elif df.loc[i, "Education"] == "2n Cycle":
            df.loc[i, "Education"] = "Master"
        
        if df.loc[i, "Marital_Status"] == "Alone":
            df.loc[i, "Marital_Status"] = "Single"
        
        elif df.loc[i, "Marital_Status"] in ["Together", "Absurd", "YOLO"]:
            df.loc[i, "Marital_Status"] = "Other"
            
        
        
    df["Age"] = date.today().year - df.Year_Birth
    df.drop(df[df.Age > 100].index, inplace=True)
    
    df.drop(df[df.Income > 200000].index, inplace=True)
    
    df["Num_Kids"] = df.Kidhome + df.Teenhome
    df.drop(["Kidhome", "Teenhome"], inplace = True, axis = 1)
    
    new_names = {"MntWines":"Wines", "MntFruits":"Fruits", "MntMeatProducts":"Meat", "MntFishProducts":"Fish",
                 "MntSweetProducts":"Sweet_Products", "MntGoldProds":"Gold_Products", "NumWebPurchases":"Web",
                "NumCatalogPurchases":"Catalog", "NumStorePurchases":"Store", "Dt_Customer":"Date_Joined"}
    df.rename(columns=new_names, inplace=True)

In [4]:
def CsvToSQL():
    
    df = LoadCsv()
    
    personal = df[["ID", "Year_Birth", "Age", "Education", "Marital_Status", "Income",
                   "Num_Kids", "Date_Joined","Complain"]]
    
    products = df[["ID", "Recency", "Wines", "Fruits", "Meat", "Fish", "Sweet_Products", "Gold_Products"]]
    
    marketing = df[["ID", "NumDealsPurchases", "AcceptedCmp1", "AcceptedCmp2", "AcceptedCmp3",
                    "AcceptedCmp4", "AcceptedCmp5", "Response"]]
    
    places = df[["ID", "Web", "Catalog", "Store", "NumWebVisitsMonth"]]
    
    personal.to_sql(
        "Personal",
        engine,
        if_exists="replace",
        index=False,
        chunksize=500
    )
    products.to_sql(
        "Products",
        engine,
        if_exists="replace",
        index=False,
        chunksize=500
    )
    marketing.to_sql(
        "Marketing",
        engine,
        if_exists="replace",
        index=False,
        chunksize=500
    )
    places.to_sql(
        "Places",
        engine,
        if_exists="replace",
        index=False,
        chunksize=500
    )
    
    sqlData = LoadDB()
    

In [5]:
def LoadDB():
    
    personalDF = pd.read_sql_table("Personal", engine)
    productsDF = pd.read_sql_table("Products", engine)
    placesDF = pd.read_sql_table("Places", engine)
    marketingDF = pd.read_sql_table("Marketing", engine)
    
    combined_df = pd.merge(personalDF, productsDF, on='ID', how='outer')
    combined_df = pd.merge(combined_df, placesDF, on='ID', how='outer')
    combined_df = pd.merge(combined_df, marketingDF, on='ID', how='outer')
    
    return combined_df
    

In [6]:
def LR_C(boolean):
    
    axisWindow = tk.Toplevel(window)
    
    xAxisLabel = tk.Label(axisWindow, text="Select X axis")
    global xAxis
    xAxis = ttk.Combobox(axisWindow, values = ["ID", "Year_Birth", "Age", "Income", "Num_Kids", "Complain",
                                               "Wines", "Fruits","Meat", "Fish", "Sweet_Products", "Gold_Products",
                                               "Recency", "Web", "Catalog","Store", "NumWebVisitsMonth",
                                               "NumDealsPurchases", "AcceptedCmp1", "AcceptedCmp2","AcceptedCmp2",
                                               "AcceptedCmp3", "AcceptedCmp4", "AcceptedCmp5", "Response"])
    xAxis.current(0)
    
    yAxisLabel = tk.Label(axisWindow, text="Select Y axis")
    global yAxis
    yAxis = ttk.Combobox(axisWindow, values = ["ID", "Year_Birth", "Age", "Income", "Num_Kids", "Complain",
                                               "Wines", "Fruits","Meat", "Fish", "Sweet_Products", "Gold_Products",
                                               "Recency", "Web", "Catalog","Store", "NumWebVisitsMonth",
                                               "NumDealsPurchases", "AcceptedCmp1", "AcceptedCmp2","AcceptedCmp2",
                                               "AcceptedCmp3", "AcceptedCmp4", "AcceptedCmp5", "Response"])
    yAxis.current(1)
    
    if boolean == True:
        plot = tk.Button(axisWindow, text="Plot", command=PlotLR)
    else:
        plot = tk.Button(axisWindow, text="Plot", command=PlotClusterElbow)
    
    xAxisLabel.pack()
    xAxis.pack()
    yAxisLabel.pack()
    yAxis.pack()
    plot.pack()

In [7]:
def PlotLR():
    
    X = xAxis.get()
    Y = yAxis.get()
    
    lrPlotWindow = tk.Toplevel(window)
    lrPlotWindow.title("Graph")
    
    fig, ax = plt.subplots()
    sns.regplot(x=X, y=Y, ax=ax, scatter_kws={'s':0.8}, data=sqlData)
    ax.set_xlabel(X)
    ax.set_ylabel(Y)
    ax.set_title("Linear Regression Plot")

    canvas = FigureCanvasTkAgg(fig, master=lrPlotWindow)
    canvas.draw()
    canvas.get_tk_widget().pack()
    

In [8]:
def PlotClusterElbow():
    
    elbowPlotWindow = tk.Toplevel(window)
    elbowPlotWindow.title("Elbow Graph")
    
    global X
    global X_
    global Y
    global Y_
    
    X = xAxis.get()
    X_ = X + "_"
    Y = yAxis.get()
    Y_ = Y + "_"
    
    global data
    data = sqlData[[X, Y]].copy()
    
    scaler = StandardScaler()    
    data[[X_, Y_]] = scaler.fit_transform(data[[X, Y]])
    
    means = []
    inertias = []
    
    for k in range(1, 10):
        kmeans = KMeans(n_clusters = k)
        kmeans.fit(data)
        means.append(k)
        inertias.append(kmeans.inertia_)
        
    fig, ax = plt.subplots()
    ax.plot(means, inertias, "o-")
    ax.set_xlabel("Num of Clusters")
    ax.set_ylabel("Inertia")
    ax.set_title("Elbow Plot")
    ax.grid(True)
    
    canvas = FigureCanvasTkAgg(fig, master = elbowPlotWindow)
    canvas.draw()
    canvas.get_tk_widget().pack()
    
    elbowText = tk.Label(elbowPlotWindow, text="Select The number of clusters for the algorithm")
    elbowText.pack()
    
    global clusters
    clusters = tk.IntVar()
    clusters.set(1)
    numberSelection = tk.OptionMenu(elbowPlotWindow, clusters, *range(1, 11))
    numberSelection.pack()    
    
    plot = tk.Button(elbowPlotWindow, text="Plot", command = PlotCluster)
    plot.pack()
    

In [9]:
def PlotCluster():
    
    clusterPlotWindow = tk.Toplevel(window)
    clusterPlotWindow.title("Graph")
    
    kmeans = KMeans(n_clusters = clusters.get())
    kmeans.fit(data[[X_, Y_]])
    data["K_Means"] = kmeans.labels_
    
    fig, ax = plt.subplots()
    ax.scatter(x = data[X], y = data[Y], c = data["K_Means"], s = 5)
    ax.set_xlabel(X)
    ax.set_ylabel(Y)
    ax.set_title("Cluster")
    
    canvas = FigureCanvasTkAgg(fig, master = clusterPlotWindow)
    canvas.draw()
    canvas.get_tk_widget().pack()
    

In [10]:
def Cl():
    
    classAxisWindow = tk.Toplevel(window)
    classAxisWindow.geometry("150x250")
    
    xAxisLabel = tk.Label(classAxisWindow, text="Select X axis")
    xAxisLabel.pack()
    global xAxis
    xAxis = tk.Listbox(classAxisWindow, selectmode=tk.MULTIPLE)
    xAxis.pack()
    
    global options
    options = ["ID", "Year_Birth", "Age", "Income", "Num_Kids", "Complain", "Wines", "Fruits",
               "Meat", "Fish", "Sweet_Products", "Gold_Products", "Recency", "Web", "Catalog",
               "Store", "NumWebVisitsMonth", "NumDealsPurchases", "AcceptedCmp1", "AcceptedCmp2",
               "AcceptedCmp2", "AcceptedCmp3", "AcceptedCmp4", "AcceptedCmp5", "Response"]
    for var in options:
        xAxis.insert(tk.END, var)
    
    yAxisLabel = tk.Label(classAxisWindow, text="Select Y axis")
    yAxisLabel.pack()
    global yAxis
    yAxis = ttk.Combobox(classAxisWindow, values = ["Education", "Marital_Status", "Num_Kids", "Complain",
                                                    "NumDealsPurchases", "AcceptedCmp1", "AcceptedCmp2",
                                                    "AcceptedCmp2", "AcceptedCmp3", "AcceptedCmp4", "AcceptedCmp5",
                                                    "Response"])
    yAxis.pack()
    
    plot = tk.Button(classAxisWindow, text="Plot", command = PlotClass)
    plot.pack()
    

In [11]:
def PlotClass():
    
    classPlotWindow = tk.Toplevel(window)
    
    selectedOptions = xAxis.curselection()
    x = [options[index] for index in selectedOptions]
    y = yAxis.get()
    
    
    X = sqlData[x]
    Y = sqlData[y]
    
    labels = Y.unique()
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state = 762)
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    
    lda = LinearDiscriminantAnalysis()
    lda.fit(X_train, Y_train)
    
    acc = tk.Label(classPlotWindow, text = "TESTING ACCURACY: {:.2f}".format(lda.score(X_test, Y_test)))
    
    actual = Y_test
    pred = lda.predict(X_test)
    
    cm = metrics.confusion_matrix(actual, pred)
    fig, ax = plt.subplots()
    ax.set_title("Classification: " + y)
    disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = labels)
    disp.plot(ax=ax)
    
    canvas = FigureCanvasTkAgg(fig, master = classPlotWindow)
    canvas.draw()
    canvas.get_tk_widget().pack()
    acc.pack()
    

In [12]:
def Hist():
    
    histAxisWindow = tk.Toplevel(window)
    
    dataLabel = tk.Label(histAxisWindow, text="Select a column")
    dataLabel.pack()
    
    global column
    column = ttk.Combobox(histAxisWindow, values = ["ID", "Year_Birth", "Age", "Education", "Marital_Status", "Income",
                                                "Num_Kids", "Date_Joined", "Complain", "Wines", "Fruits", "Meat",
                                                "Fish", "Sweet_Products", "Gold_Products", "Recency", "Web", "Catalog",
                                                "Store", "NumWebVisitsMonth", "NumDealsPurchases", "AcceptedCmp1",
                                                "AcceptedCmp2","AcceptedCmp2", "AcceptedCmp3", "AcceptedCmp4",
                                                "AcceptedCmp5", "Response"])
    column.pack()
    
    plot = tk.Button(histAxisWindow, text="Plot", command = PlotHist)
    plot.pack()
    

In [13]:
def PlotHist():
    
    histPlotWindow = tk.Toplevel(window)
    
    x = column.get()
    X = sqlData[x]
    
    fig, ax = plt.subplots()
    ax.hist(X, edgecolor = "black")
    ax.set_xlabel(x)
    ax.set_ylabel("Frequency")
    ax.set_title("Histogram")
    
    canvas = FigureCanvasTkAgg(fig, master = histPlotWindow)
    canvas.draw()
    canvas.get_tk_widget().pack()
    

In [14]:
def SelectNextWindow():
    selection = alg.get()
    
    if selection == "Linear Regression":
        LR_C(True)
    elif selection == "Clustering":
        LR_C(False)
    elif selection == "Classification":
        Cl()
    elif selection == "Histogram":
        Hist()

In [15]:
#__RUN THIS TO START THE PROGRAM__

window = tk.Tk()
window.title("Customer Details & Habits")
window.geometry("200x100")

engine = create_engine("sqlite:///Marketing_Campaign_Data.db", echo = False)
global sqlData
sqlData = LoadDB()

alg = ttk.Combobox(window, values = ["Linear Regression", "Clustering", "Classification", "Histogram"])
alg.current(0)
alg.pack()

plot_button = tk.Button(window, text="Select Graph Type", command=SelectNextWindow)
plot_button.pack()

updateDB = tk.Button(window, text="Update Database With CSV", command=CsvToSQL)
updateDB.pack()

window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Samuel\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Samuel\AppData\Local\Temp\ipykernel_19716\311908622.py", line 27, in PlotClusterElbow
    kmeans.fit(data)
  File "C:\Users\Samuel\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Samuel\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py", line 1471, in fit
    X = self._validate_data(
        ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Samuel\anaconda3\Lib\site-packages\sklearn\base.py", line 579, in _validate_data
    self._check_feature_names(X, reset=reset)
  File "C:\Users\Samuel\anaconda3\Lib\site-packages\sklearn\base.py", line 440, in _check_feature_names
    feature_names_in = _get_feature_names(X)
                       ^^^^^^^